## **作业：实现通用的批量梯度下降算法模块**
根据函数$y = 3 {x_1}^2 - 5 x_1 x_2 + 2 {x_2}^2 - 8 x_1 - 10 x_2 + 6$生成若干原始数据点,  
要求使用梯度下降算法拟合出函数：$h(w, x) = w_5 {x_1}^2 + w_4 x_1 x_2 + w_3 {x_2}^2 + w_2 x_1 + w_1 x_2 + w_0$的系数$w$，  
使得该函数与上述原始数据点具有最小二乘解。

### **任务1：了解待拟合的数据源**
下面的代码生成了待拟合的数据源，变量x1和x2形成网格数据点，共36个

In [1]:
import numpy as np

# 用于生成样本数据点的函数 
def f(x1, x2):
    # np.random.random()是为了给样本数据一个小的扰动
    return 3 * x1 * x1 - 5 * x1 * x2 + 2 * x2 * x2 - 8 * x1 - 10 * x2 + 6 + np.random.random()

# 给定一批原始数据点的x1和x2两个方向坐标
x1 = np.linspace(-3, 3, 6)
x2 = np.linspace(-3, 3, 6)
y = np.zeros(len(x1) * len(x2))
# 根据x1和x2中的每个组合，分别计算对应的y值
y_index = 0
for i in np.arange(len(x1)):
    for j in np.arange(len(x2)):
        y[y_index] = f(x1[i], x2[j])
        y_index += 1
        # 打印每个数据点的坐标（x1, x2, y)
        print("#{0}, ({1:.2f},{2:.2f}),{3:.2f}".format(y_index, x1[i], x2[j], y[y_index-1]))

#1, (-3.00,-3.00),60.63
#2, (-3.00,-1.80),55.18
#3, (-3.00,-0.60),54.84
#4, (-3.00,0.60),61.03
#5, (-3.00,1.80),73.38
#6, (-3.00,3.00),90.36
#7, (-1.80,-3.00),52.08
#8, (-1.80,-1.80),38.79
#9, (-1.80,-0.60),32.12
#10, (-1.80,0.60),30.39
#11, (-1.80,1.80),34.84
#12, (-1.80,3.00),45.79
#13, (-0.60,-3.00),51.61
#14, (-0.60,-1.80),30.98
#15, (-0.60,-0.60),17.28
#16, (-0.60,0.60),9.28
#17, (-0.60,1.80),6.12
#18, (-0.60,3.00),9.84
#19, (0.60,-3.00),59.31
#20, (0.60,-1.80),32.54
#21, (0.60,-0.60),11.74
#22, (0.60,0.60),-4.38
#23, (0.60,1.80),-14.35
#24, (0.60,3.00),-18.16
#25, (1.80,-3.00),76.80
#26, (1.80,-1.80),42.52
#27, (1.80,-0.60),14.12
#28, (1.80,0.60),-9.20
#29, (1.80,1.80),-26.23
#30, (1.80,3.00),-37.45
#31, (3.00,-3.00),102.78
#32, (3.00,-1.80),60.69
#33, (3.00,-0.60),25.42
#34, (3.00,0.60),-5.23
#35, (3.00,1.80),-29.01
#36, (3.00,3.00),-47.05


### **任务2：了解扩展的自变量维度矩阵**
下面的代码将任务1中36组数据点$(x_1,x_2) \to y$扩展成：$({x_1}^2, x_1 x_2, {x_2}^2, x_1, x_2, 1) \to y$的形式  
维度矩阵包含6列的数据：

In [2]:
POLY_COUNT = 6                  # 2阶多项式一共6项
def map_data(X1, X2):           # 生成两个变量的多元一阶矩阵
    variables = np.ones((len(X1) * len(X2), POLY_COUNT))
    y = np.zeros((len(X1) * len(X2)))
    row_index = 0
    for i in np.arange(len(X1)):
        for j in np.arange(len(X2)):
            row = variables[row_index]
            row[0] = X1[i] * X1[i]
            row[1] = X1[i] * X2[j]
            row[2] = X2[j] * X2[j]
            row[3] = X1[i]
            row[4] = X2[j]
            y[row_index] = f(X1[i], X2[j])
            #row[5] = 1
            row_index += 1
    return (variables, y)

x_ext, y = map_data(x1, x2)
print("X维度矩阵：")
print(x_ext)

X维度矩阵：
[[ 9.    9.    9.   -3.   -3.    1.  ]
 [ 9.    5.4   3.24 -3.   -1.8   1.  ]
 [ 9.    1.8   0.36 -3.   -0.6   1.  ]
 [ 9.   -1.8   0.36 -3.    0.6   1.  ]
 [ 9.   -5.4   3.24 -3.    1.8   1.  ]
 [ 9.   -9.    9.   -3.    3.    1.  ]
 [ 3.24  5.4   9.   -1.8  -3.    1.  ]
 [ 3.24  3.24  3.24 -1.8  -1.8   1.  ]
 [ 3.24  1.08  0.36 -1.8  -0.6   1.  ]
 [ 3.24 -1.08  0.36 -1.8   0.6   1.  ]
 [ 3.24 -3.24  3.24 -1.8   1.8   1.  ]
 [ 3.24 -5.4   9.   -1.8   3.    1.  ]
 [ 0.36  1.8   9.   -0.6  -3.    1.  ]
 [ 0.36  1.08  3.24 -0.6  -1.8   1.  ]
 [ 0.36  0.36  0.36 -0.6  -0.6   1.  ]
 [ 0.36 -0.36  0.36 -0.6   0.6   1.  ]
 [ 0.36 -1.08  3.24 -0.6   1.8   1.  ]
 [ 0.36 -1.8   9.   -0.6   3.    1.  ]
 [ 0.36 -1.8   9.    0.6  -3.    1.  ]
 [ 0.36 -1.08  3.24  0.6  -1.8   1.  ]
 [ 0.36 -0.36  0.36  0.6  -0.6   1.  ]
 [ 0.36  0.36  0.36  0.6   0.6   1.  ]
 [ 0.36  1.08  3.24  0.6   1.8   1.  ]
 [ 0.36  1.8   9.    0.6   3.    1.  ]
 [ 3.24 -5.4   9.    1.8  -3.    1.  ]
 [ 3.24 -3.24  3.2

### **任务3：阅读理解通用的批量梯度下降算法函数**

In [3]:
def batch_gradient_descent(target_fn, gradient_fn, init_W, X, Y, learning_rate=0.001, tolerance=1e-7):
    """支持多变量的批量梯度下降法"""
    # 假设函数为：y = wn * xn + w(n-1) * x(n-1) +... + w2 * x2 + w1 * x1 + w0 * x0 其中，x0为1
    # X中：第一列为xn,第二列为x(n-1)，依次类推，最后一列为x0(全为1)
    # W向量顺序是：wn,w(n-1),...w1,w0，要确保与X中各列顺序一致
    W = init_W
    target_value = target_fn(W, X, Y) 
    iter_count = 0
    while iter_count < 50000:                      # 如果50000次循环仍未收敛，则认为无法收敛
        gradient = gradient_fn(W, X, Y)
        next_W = W - gradient * learning_rate 
        next_target_value = target_fn(next_W, X, Y)
        if abs(target_value - next_target_value) < tolerance:
            print("循环", iter_count, "次后收敛")
            return W
        else:                                             
            W, target_value = next_W, next_target_value
            iter_count += 1
    
    print("50000次循环后，计算仍未收敛")
    return W

### **任务4：编写目标函数、梯度函数，并调用批量梯度下降函数完成拟合**
* 定位下面的代码框架中所有的"### TODO"，按照提示完成代码编写
 * 编写完成target_function
 * 编写完成gradient_function
 * 调整learning_rate的值使批量梯度下降计算较快较好的收敛
 * 编写完成batch_gradient_descent的函数调用
* 运行代码，观察拟合出来的系数，应该与3, -5, 2, -8, 10, 6接近

In [ ]:
### TODO:实现下列目标函数（按最小二乘法）
def target_function(W, X, Y):                           # 修改return返回值
    return 0.0

### TODO:实现下列梯度函数（返回梯度数组）
def gradient_function(W, X, Y):                         # 修改return返回值
    return 0.0

np.random.seed(0)
init_W = np.random.randn(x_ext.shape[1])                # x_ext现在是具有6个列的矩阵，因此init_W需要6个初始元素

### TODO:调整learning_rate使批量梯度下降计算较快较好的收敛
learning_rate = 0.05                                    # 可调整learning_rate

tolerance = 1e-7                                        # 可调整tolerance

### TODO：调用batch_gradient_descent计算拟合系数，请填充其参数
W = batch_gradient_descent(# 填充参数 #)

print("W值", W)                                         
print("拟合曲线计算的数据值与真实值的差异：", x_ext.dot(W) - y)